# CLIMATE & WEATHER ANALYSIS

In [1]:
%run ./_setup.py
import os
import sys


Python: c:\Users\Admin\Desktop\Irrigation Water Requirement Prediction\.venv\Scripts\python.exe
Environment ready
Shape: (10000, 20)
<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Soil_Type                10000 non-null  str    
 1   Soil_pH                  10000 non-null  float64
 2   Soil_Moisture            10000 non-null  float64
 3   Organic_Carbon           10000 non-null  float64
 4   Electrical_Conductivity  10000 non-null  float64
 5   Temperature_C            10000 non-null  float64
 6   Humidity                 10000 non-null  float64
 7   Rainfall_mm              10000 non-null  float64
 8   Sunlight_Hours           10000 non-null  float64
 9   Wind_Speed_kmh           10000 non-null  float64
 10  Crop_Type                10000 non-null  str    
 11  Crop_Growth_Stage        10000 non-null  str    
 12  Season   

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
df = pd.read_csv("../data/irrigation_prediction.csv")

In [12]:
# Color palette
season_colors = {'Rabi': '#FF6B6B', 'Kharif': '#4ECDC4', 'Zaid': '#45B7D1'}
weather_colors = px.colors.sequential.Viridis
crop_colors = px.colors.qualitative.Set3[:len(df['Crop_Type'].unique())]

region_color_map = {
    "North": "#1f77b4",
    "South": "#ff7f0e",
    "East":  "#2ca02c",
    "West":  "#d62728"
}


crop_color_map = {'Wheat': '#FFD700', 'Maize': '#32CD32', 'Cotton': "#E917C6", 
                  'Rice': '#87CEEB', 'Sugarcane': '#D2691E', 'Potato': '#8B0000'}



### *46. Temperature × Humidity Comfort Zones*

In [30]:
print("\n46. Crop Comfort Zones...")

fig46 = px.density_contour(df, x='Temperature_C', y='Humidity',
                          color='Crop_Type', facet_col='Crop_Type',
                          facet_col_wrap=3, title='<b>46. Crop-Specific Comfort Zones</b><br>Temperature vs Humidity Ranges',
                          labels={'Temperature_C': 'Temperature (°C)', 'Humidity': 'Humidity (%)'},
                          color_discrete_sequence=crop_colors)

fig46.update_layout(height=700, plot_bgcolor='black')
fig46.show()


46. Crop Comfort Zones...


### *47. Seasonal Rainfall Patterns*

In [29]:
print("\n47. Seasonal Rainfall Patterns...")

rain_season = df.groupby(['Region', 'Season'], observed=False).agg(
    Avg_Rainfall=('Rainfall_mm', 'mean'),
    Rain_Variability=('Rainfall_mm', 'std'),
    Count=('Season', 'count')
).reset_index()

fig47 = px.box(df, x='Season', y='Rainfall_mm', color='Region',
               title='<b>47. Rainfall Distribution by Season and Region</b>',
               labels={'Rainfall_mm': 'Rainfall (mm)', 'Season': 'Season'},
               color_discrete_map=region_color_map)

fig47.update_layout(height=500, plot_bgcolor='black', boxmode='group')
fig47.show()


47. Seasonal Rainfall Patterns...


### *48. Sunlight Hours vs Growth*

In [28]:
print("\n48. Sunlight Impact Analysis...")

sunlight_impact = df.groupby(['Crop_Type', 'Crop_Growth_Stage'], observed=False).agg(
    Avg_Sunlight=('Sunlight_Hours', 'mean'),
    Avg_Moisture=('Soil_Moisture', 'mean'),
    Success_Rate=('Irrigation_Need', lambda x: (x == 'Low').mean() * 100)
).reset_index()

fig48 = px.scatter(sunlight_impact, x='Avg_Sunlight', y='Success_Rate',
                   color='Crop_Type', size='Avg_Moisture',
                   facet_col='Crop_Growth_Stage', facet_col_wrap=2,
                   title='<b>48. Sunlight Impact on Success Rate</b><br>Size = Soil Moisture',
                   labels={'Avg_Sunlight': 'Avg Sunlight Hours', 'Success_Rate': 'Low Irrigation Need (%)'},
                   color_discrete_map=crop_color_map)

fig48.update_layout(height=600, plot_bgcolor='black')
fig48.show()


48. Sunlight Impact Analysis...


### *49. Wind Speed Impact*

In [14]:
print("\n49. Wind Speed Impact...")

df['Evaporation_Risk'] = df['Wind_Speed_kmh'] * df['Temperature_C'] / 100

fig49 = px.scatter(df, x='Wind_Speed_kmh', y='Soil_Moisture',
                   color='Evaporation_Risk', size='Temperature_C',
                   facet_col='Irrigation_Type', facet_col_wrap=2,
                   title='<b>49. Wind Impact on Soil Moisture</b><br>Color = Evaporation Risk, Size = Temperature',
                   labels={'Wind_Speed_kmh': 'Wind Speed (km/h)', 'Soil_Moisture': 'Soil Moisture (%)'},
                   color_continuous_scale='RdYlBu_r')

fig49.update_layout(height=600, plot_bgcolor='black')
fig49.show()


49. Wind Speed Impact...


### *50. Climate Trend Analysis*

In [27]:
from plotly.subplots import make_subplots

print("\n50. Climate Trend Analysis...")

# Create time proxy (assume data has some temporal order)
df_sorted = df.sort_values(['Region', 'Season']).reset_index(drop=True)
df_sorted['Time_Sequence'] = range(len(df_sorted))

# Calculate rolling averages
window = min(200, len(df_sorted)//20)
df_sorted['Temp_MA'] = df_sorted['Temperature_C'].rolling(window=window, center=True).mean()
df_sorted['Rain_MA'] = df_sorted['Rainfall_mm'].rolling(window=window, center=True).mean()

fig50 = make_subplots(rows=2, cols=1, subplot_titles=('Temperature Trend', 'Rainfall Trend'))

fig50.add_trace(go.Scatter(x=df_sorted['Time_Sequence'], y=df_sorted['Temp_MA'],
                          mode='lines', name='Temperature', line=dict(color='red')),
                row=1, col=1)

fig50.add_trace(go.Scatter(x=df_sorted['Time_Sequence'], y=df_sorted['Rain_MA'],
                          mode='lines', name='Rainfall', line=dict(color='blue')),
                row=2, col=1)

fig50.update_layout(height=600, title_text='<b>50. Climate Trends Over Time</b>', plot_bgcolor='black')
fig50.show()


50. Climate Trend Analysis...


### 51. Microclimate Variations

In [26]:
print("\n51. Microclimate Variations...")

microclimate = df.groupby(['Region', 'Soil_Type'], observed=False).agg({
    'Temperature_C': ['mean', 'std'],
    'Humidity': ['mean', 'std'],
    'Rainfall_mm': 'mean'
}).round(2)
microclimate.columns = ['Temp_Mean', 'Temp_Std', 'Humidity_Mean', 'Humidity_Std', 'Rainfall_Mean']
microclimate = microclimate.reset_index()

fig51 = px.scatter(microclimate, x='Temp_Mean', y='Humidity_Mean',
                   color='Region', size='Rainfall_Mean',
                   error_x='Temp_Std', error_y='Humidity_Std',
                   title='<b>51. Microclimate Variations by Region & Soil</b><br>Error bars show variability',
                   labels={'Temp_Mean': 'Avg Temperature (°C)', 'Humidity_Mean': 'Avg Humidity (%)'},
                   color_discrete_map=region_color_map)

fig51.update_layout(height=500, plot_bgcolor='black')
fig51.show()


51. Microclimate Variations...


### *52. Weather Extremes Frequency*

In [25]:
print("\n52. Weather Extremes Analysis...")

# Define extremes
df['Heat_Extreme'] = df['Temperature_C'] > 35
df['Rain_Extreme'] = df['Rainfall_mm'] > 100
df['Drought'] = df['Rainfall_mm'] < 10

extremes = df.groupby('Season', observed=False).agg({
    'Heat_Extreme': 'mean',
    'Rain_Extreme': 'mean',
    'Drought': 'mean'
}).reset_index()

fig52 = px.bar(extremes.melt(id_vars='Season'), x='Season', y='value',
               color='variable', barmode='group',
               title='<b>52. Frequency of Weather Extremes by Season</b>',
               labels={'value': 'Frequency (%)', 'variable': 'Extreme Type'},
               color_discrete_map={'Heat_Extreme': 'red', 'Rain_Extreme': 'blue', 'Drought': 'orange'})

fig52.update_layout(height=500, plot_bgcolor='black')
fig52.show()


52. Weather Extremes Analysis...


### *53. Evapotranspiration Analysis*

In [24]:
print("\n53. Evapotranspiration Analysis...")

# Simplified ET calculation (Hargreaves)
df['ET_estimate'] = 0.0023 * (df['Temperature_C'] + 17.8) * (df['Temperature_C'].max() - df['Temperature_C']) ** 0.5 * df['Sunlight_Hours']

fig53 = px.scatter(df, x='ET_estimate', y='Soil_Moisture',
                   color='Crop_Type', facet_col='Season',
                   title='<b>53. Evapotranspiration vs Soil Moisture</b>',
                   labels={'ET_estimate': 'Estimated ET (mm/day)', 'Soil_Moisture': 'Soil Moisture (%)'},
                   color_discrete_map=crop_color_map)

fig53.update_layout(height=500, plot_bgcolor='black')
fig53.show()


53. Evapotranspiration Analysis...


### *54. Frost Risk Periods*

In [23]:
print("\n54. Frost Risk Analysis...")

df['Frost_Risk'] = df['Temperature_C'] < 5
frost_by_season = df.groupby(['Season', 'Region'], observed=False)['Frost_Risk'].mean().reset_index()

fig54 = px.bar(frost_by_season, x='Season', y='Frost_Risk',
               color='Region', barmode='group',
               title='<b>54. Frost Risk by Season and Region</b>',
               labels={'Frost_Risk': 'Probability of Frost', 'Season': 'Season'},
               color_discrete_map=region_color_map)

fig54.update_layout(height=500, plot_bgcolor='black')
fig54.show()


54. Frost Risk Analysis...


### *55. Heat Stress Days*

In [22]:
print("\n55. Heat Stress Analysis...")

heat_thresholds = {'Wheat': 32, 'Maize': 35, 'Cotton': 38, 'Rice': 36, 'Sugarcane': 40, 'Potato': 30}
df['Crop_Heat_Threshold'] = df['Crop_Type'].map(heat_thresholds)
df['Heat_Stress'] = df['Temperature_C'] > df['Crop_Heat_Threshold']

heat_stress = df.groupby(['Region', 'Crop_Type'], observed=False).agg(
    Heat_Stress_Days=('Heat_Stress', 'mean'),
    Avg_Temp=('Temperature_C', 'mean')
).reset_index()

fig55 = px.scatter(heat_stress, x='Avg_Temp', y='Heat_Stress_Days',
                   color='Crop_Type', size='Heat_Stress_Days',
                   facet_col='Region', facet_col_wrap=2,
                   title='<b>55. Heat Stress Vulnerability</b>',
                   labels={'Avg_Temp': 'Avg Temperature (°C)', 'Heat_Stress_Days': 'Heat Stress Frequency'},
                   color_discrete_map=crop_color_map)

fig55.update_layout(height=600, plot_bgcolor='black')
fig55.show()


55. Heat Stress Analysis...


### *56. Rainfall Intensity Distribution*

In [31]:
print("\n56. Rainfall Intensity Analysis...")

df['Rain_Intensity'] = pd.cut(df['Rainfall_mm'], 
                             bins=[0, 10, 25, 50, 100, 500],
                             labels=['Very Light', 'Light', 'Moderate', 'Heavy', 'Very Heavy'])

intensity_dist = df.groupby(['Season', 'Rain_Intensity'], observed=False).size().reset_index(name='Count')

fig56 = px.bar(intensity_dist, x='Season', y='Count',
               color='Rain_Intensity', barmode='stack',
               title='<b>56. Rainfall Intensity Distribution by Season</b>',
               labels={'Count': 'Number of Cases', 'Season': 'Season'},
               color_discrete_sequence=px.colors.sequential.Blues)

fig56.update_layout(height=500, plot_bgcolor='black')
fig56.show()


56. Rainfall Intensity Analysis...


### *57. Growing Degree Days*

In [33]:
print("\n57. Growing Degree Days Analysis...")

base_temp = 10
df['GDD'] = (df['Temperature_C'] - base_temp).clip(lower=0)

gdd_accumulation = df.groupby(['Crop_Type', 'Crop_Growth_Stage'], observed=False)['GDD'].mean().reset_index()

fig57 = px.line(gdd_accumulation, x='Crop_Growth_Stage', y='GDD',
                color='Crop_Type', markers=True,
                title='<b>57. Growing Degree Days by Crop and Growth Stage</b>',
                labels={'GDD': 'Growing Degree Days', 'Crop_Growth_Stage': 'Growth Stage'},
                color_discrete_map=crop_color_map)

fig57.update_layout(height=500, plot_bgcolor='black',
                   xaxis={'categoryorder': 'array',
                          'categoryarray': ['Sowing', 'Vegetative', 'Flowering', 'Harvest']})
fig57.show()


57. Growing Degree Days Analysis...


### *58. Climate Similarity Analysis*

In [34]:
print("\n58. Climate Similarity Regions...")

climate_profile = df.groupby('Region', observed=False).agg({
    'Temperature_C': 'mean',
    'Rainfall_mm': 'mean',
    'Humidity': 'mean'
}).reset_index()

fig58 = px.scatter(climate_profile, x='Temperature_C', y='Rainfall_mm',
                   color='Region', size='Humidity',
                   title='<b>58. Climate Similarity Between Regions</b><br>Size = Avg Humidity',
                   labels={'Temperature_C': 'Avg Temperature (°C)', 'Rainfall_mm': 'Avg Rainfall (mm)'},
                   color_discrete_map=region_color_map)

fig58.update_layout(height=500, plot_bgcolor='black')
fig58.show()


58. Climate Similarity Regions...


### *59. Weather Forecast Accuracy*

In [36]:
print("\n59. Weather Prediction Analysis...")

# Create synthetic forecast error (for demonstration)
np.random.seed(42)
df['Forecast_Temp'] = df['Temperature_C'] + np.random.normal(0, 2, len(df))
df['Forecast_Rain'] = df['Rainfall_mm'] * (1 + np.random.normal(0, 0.3, len(df))).clip(0.5, 1.5)
df['Temp_Error'] = abs(df['Forecast_Temp'] - df['Temperature_C'])
df['Rain_Error'] = abs(df['Forecast_Rain'] - df['Rainfall_mm'])

error_by_season = df.groupby('Season', observed=False).agg({
    'Temp_Error': 'mean',
    'Rain_Error': 'mean'
}).reset_index()

fig59 = px.bar(error_by_season.melt(id_vars='Season'), x='Season', y='value',
               color='variable', barmode='group',
               title='<b>59. Weather Forecast Accuracy by Season</b>',
               labels={'value': 'Average Error', 'variable': 'Forecast Type'},
               color_discrete_map={'Temp_Error': 'orange', 'Rain_Error': 'blue'})

fig59.update_layout(height=500, plot_bgcolor='black')
fig59.show()


59. Weather Prediction Analysis...


### *60. Climate Resilience Index*

In [38]:
print("\n60. Climate Resilience Analysis...")

# Calculate resilience score
df['Temp_Resilience'] = 100 - abs(df['Temperature_C'] - 25)  # Optimal around 25°C
df['Rain_Resilience'] = df['Rainfall_mm'].clip(0, 100)  # 0-100mm is good
df['Humidity_Resilience'] = 100 - abs(df['Humidity'] - 65)  # Optimal around 65%

df['Climate_Resilience'] = (df['Temp_Resilience'] + df['Rain_Resilience'] + df['Humidity_Resilience']) / 3

resilience_by_region = df.groupby('Region', observed=False).agg({
    'Climate_Resilience': 'mean',
    'Temp_Resilience': 'mean',
    'Rain_Resilience': 'mean',
    'Humidity_Resilience': 'mean'
}).reset_index()

fig60 = px.bar(resilience_by_region.melt(id_vars='Region', 
                                        value_vars=['Temp_Resilience', 'Rain_Resilience', 'Humidity_Resilience']),
               x='Region', y='value', color='variable', barmode='group',
               title='<b>60. Climate Resilience Components by Region</b>',
               labels={'value': 'Resilience Score (0-100)', 'variable': 'Climate Factor'},
               color_discrete_sequence=['#FF6B6B', '#4ECDC4', "#ACF721"])

fig60.update_layout(height=500, plot_bgcolor='black')
fig60.show()


60. Climate Resilience Analysis...
